In [1]:
import os

import numpy as np
from pprint import pprint
from tqdm import tqdm
from itertools import product

import plotly as py
import plotly.graph_objects as go

In [2]:
os.chdir('../')
print(os.getcwd())

#if not os.path.exists('visualisations'):
#    os.mkdir('visualisations')

visualisations_dir = os.getcwd() + '\\visualisations'
print(f'Saving to visualisations dir: {visualisations_dir}')

from Tetrahedron_Cevian_Plotting import TetrahedronCevianPlotting

c:\Users\subst\Projects\GitBase\Tetrahedron-Cevian-Plotting
Saving to visualisations dir: c:\Users\subst\Projects\GitBase\Tetrahedron-Cevian-Plotting\visualisations


In [3]:
import warnings
from collections import namedtuple, deque

import numpy as np


class LogisticMapGenerator:

    ValueRange = namedtuple('ValueRange', 'min max')

    # Global Params
    depth_range = ValueRange(1, 32)
    r_range = ValueRange(0.0, 4.0)
    map_range = ValueRange(0.0, 1.0)

    valid_ret_types = ('alpha', 'decimal', 'ternary')

    def __init__(self,
                 x: np.float,
                 r: np.float,
                 alphabet: str,
                 depth: int,
                 ret_type: str,
                 ret_history: int = 2,
                 throwaway_itts: int = 10000
                 ):

        self.alphabet = alphabet
        self.depth = depth
        self.r = r

        self.ret_type = ret_type
        self.ret_history = ret_history

        self.brackets = len(self.alphabet) + 1

        self.x_vals = np.array([x], dtype=np.float64)
        if self.ret_type == 'alpha':
            self.labels = np.array([self.__get_label(x)], dtype=str)

        self.return_lookups = {'alpha': lambda: self.labels,
                               'decimal': lambda: np.array(self.x_vals),
                               'ternary': lambda: self.__evaluate_ternary()
                               }

        for _ in range(throwaway_itts):
            _ = next(self)

    @property
    def ret_type(self):
        return self.__ret_type

    @ret_type.setter
    def ret_type(self, ret_type):
        if ret_type not in self.valid_ret_types:
            warn = f'Valid Return Types are: {self.valid_ret_types}.\
            Default return type used. ret_type set to "alpha".'
            warnings.warn(warn, UserWarning)
            self.__ret_type = 'alpha'
        else:
            self.__ret_type = ret_type

    @property
    def ret_history(self):
        return self.__ret_history

    @ret_history.setter
    def ret_history(self, ret_history):
        if ret_history <= 0:
            warn = f'ret_history must be > 0.\
            ret_hist set to 2.'
            warnings.warn(warn, UserWarning)
            self.__ret_history = 2

        elif (self.ret_type == 'ternary') and (ret_history != 3):
            warn = f'ret_history must be = 3 for ret_type = "ternary".\
            ret_hist set to 3.'
            warnings.warn(warn, UserWarning)
            self.__ret_history = 3

        else:
            self.__ret_history = ret_history

    @property
    def r(self):
        return self.__r

    @r.setter
    def r(self, r):
        if self.r_range.min < r < self.r_range.max:
            self.__r = r
        else:
            error = f'r-value must be in the range \
            ({self.r_range.min} - f{self.r_range.max}).'
            raise ValueError(error)

    @property
    def depth(self):
        return self.__depth

    @depth.setter
    def depth(self, depth):
        if self.depth_range.min <= depth <= self.depth_range.max:
            self.__depth = int(depth)
        else:
            error = f'depth must be in the range \
            [{self.depth_range.min} - {self.depth_range.max}].'
            raise ValueError(error)

    def __get_label(self, x):

        # Helper function -> get brackets w/ endpoints
        # & appropriate brackets count
        def get_brackets(start, stop):
            return np.linspace(start=start,
                               stop=stop,
                               num=self.brackets,
                               endpoint=True)

        # set initial search brackets
        search_brackets = get_brackets(start=self.map_range.min,
                                       stop=self.map_range.max)

        # squence representation of decimal value
        sequence = ''
        for _ in range(self.depth):
            # find first index larger than input value
            # -> new search brackets = [value_index: value_index+1]
            value_index = np.argmax(search_brackets > x) - 1

            # Update sequence w/ new state
            sequence += self.alphabet[value_index]

            min_lim = search_brackets[value_index]
            max_lim = search_brackets[value_index + 1]
            search_brackets = get_brackets(start=min_lim,
                                           stop=max_lim)

        return sequence

    def __evaluate_ternary(self):
        return np.array(self.x_vals) / np.sum(self.x_vals)

    def __evaluate_map(self):
        last_x = self.x_vals[-1]
        return last_x * (1 - last_x) * self.r

    def __next__(self):
        
        while len(self.x_vals) <= self.ret_history:
            next_x = self.__evaluate_map()
            self.x_vals= np.append(self.x_vals, next_x)
            
            if self.ret_type == 'alpha':
                next_label = self.__get_label(next_x)
                self.labels = np.append(self.labels, next_label)
                
        if len(self.x_vals) >= self.ret_history:
            self.x_vals = self.x_vals[1:]
            if self.ret_type == 'alpha':
                self.labels = self.labels[1:]

        return self.return_lookups[self.ret_type]()

In [4]:


logi_gen = LogisticMapGenerator(x = np.random.rand(),
                               r = 1.2,
                               alphabet = 'ABCD',
                               depth = 6,
                               ret_type = 'decimal',
                               ret_history = 1,
                               throwaway_itts = 1024
                              )

traces_count = 196
itterations_count = 4096
plot_traces = []
pdata = None
labels= []
for r_val in tqdm(np.linspace(start=2.9, stop=4.0, num=traces_count, endpoint=False), total=traces_count):

    logi_gen = LogisticMapGenerator(x = np.random.rand(),
                                   r = r_val,
                                   alphabet = 'ABCD',
                                   depth = 6,
                                   ret_type = 'decimal',
                                   ret_history = 1,
                                   throwaway_itts = 512
                                  )
    data = np.array([next(logi_gen) for _ in range(itterations_count)])
    
    labels.append(r_val)
    if not isinstance(pdata, np.ndarray):
        pdata = data
    else:
        pdata= np.append(pdata, data, axis=1)

100%|████████████████████████████████████████████████████████████████████████████████| 196/196 [00:07<00:00, 24.87it/s]


In [5]:
data_convertor = TetrahedronCevianPlotting(ptype='triangle',
                                           fname='2D Triangle Scatter Plot [t=0, t=+1, t=+2] of Logistic Map')
plotting_values = data_convertor.plot_data(pdata, labels=labels)

data_convertor = TetrahedronCevianPlotting(ptype='tetrahedron',
                                           fname='3D Tetrahedron Scatter Plot [t=0, t=+1, t=+2, t=+3] of Logistic Map')
data_convertor.plot_data(pdata, labels=labels)

Processing Traces:


100%|████████████████████████████████████████████████████████████████████████████████| 196/196 [00:20<00:00,  9.75it/s]


Generating Traces: 


100%|██████████████████████████████████████████████████████████████████████████████| 196/196 [00:00<00:00, 2241.24it/s]


Writing Plot to disk at: c:\Users\subst\Projects\GitBase\Tetrahedron-Cevian-Plotting\visualisations\2D Triangle Scatter Plot [t=0, t=+1, t=+2] of Logistic Map.html
Processing Traces:


100%|████████████████████████████████████████████████████████████████████████████████| 196/196 [00:29<00:00,  6.61it/s]


Generating Traces: 


100%|██████████████████████████████████████████████████████████████████████████████| 196/196 [00:00<00:00, 1259.88it/s]


Writing Plot to disk at: c:\Users\subst\Projects\GitBase\Tetrahedron-Cevian-Plotting\visualisations\3D Tetrahedron Scatter Plot [t=0, t=+1, t=+2, t=+3] of Logistic Map.html
